In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from dataloader import DataGenerator
from models import *
from utils import show_history
from callbacks import EarlyStoppingReduceLROnPlateau

In [2]:
seed = 2000

In [3]:
tf.keras.utils.set_random_seed(
    seed
)
np.random.default_rng(seed)
tf.config.experimental.enable_op_determinism()

In [4]:
params = {
    'batch_size': 1,
    'shape': (128, 128, 128),
    'n_channels': 1,
    'shuffle': True
}

train_path = "data/train/"
val_path = "data/validation/"
save_path = 'models/'

train_generator = DataGenerator(path=train_path, ids=range(200), **params)
valid_generator = DataGenerator(path=val_path, ids=range(20), **params)

In [5]:
callbacks = EarlyStoppingReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, min_delta=1e-4,
                                           restore_best_weights=True, early_stopping=True, verbose=1)

In [ ]:
model = inception_unet(seed=seed)
model.summary()
print(f'=============Training {model.name} model===============')
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='bce',
              metrics=['binary_accuracy'])

# Fit the model
history = model.fit(train_generator, validation_data=valid_generator,
                    epochs=100,
                    callbacks=[callbacks])

model.save_weights(save_path + model.name + '.h5')
print(f'Saved {model.name} model weights')

In [ ]:
show_history(history)